# KENDAMA RL PROJECT

### Visulalisation of the environment

If you execute the cell below, you will discover how the Kendama environment works. You can even change the action you want to give to the algorithm by modify the array action. You can also directly drag the ken and the dama on the simulation with your mouse. Have fun !

#### Be careful : You need to reload the entire notebook if you closed the simulation's window and yet want to execute the simulation again

In [ ]:
from IPython.display import clear_output
from kendama_env import KendamaEnv
import numpy as np 
import time
env = KendamaEnv()

a = 0
start_time = time.time()
while 1:
    current_time = time.time()
    elapsed_time = current_time - start_time

    if elapsed_time > 1.0/240:
        a += 0.005
        start_time = time.time()
        # Change this if you want to give actions to the ken
        action = [0,0,0.7*np.sin(10*a),0,0,0]
        
        clear_output(wait=True)
        
        ob, reward, done, _ = env.step(action)

        if(done):
            env.reset()

# Loading, Training and Evaluating the agents


## PP02 agent
This cell lets you load the PPO2 agent we trained (on 5 millions iterations number).

In [ ]:
import gym
from IPython.display import clear_output
from kendama_env import KendamaEnv
import numpy as np 
import time
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines import PPO2,SAC
from stable_baselines.common.vec_env import VecNormalize
from IPython.display import clear_output

def make_env(rank, seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = KendamaEnv(render=(rank==0))
        return env
    
    return _init

if __name__ == '__main__':
    # Uncomment this and comment below if you want to use multiprocessing
    #num_cpu = 4
    #env = SubprocVecEnv([make_env( i) for i in range(num_cpu)])
    #env = VecNormalize(env, norm_reward= False)
    env = KendamaEnv(render=True)
    env = DummyVecEnv([lambda:env])
    
    env = VecNormalize.load("saves/vec_normalize_ppo.pkl", env)
    env.norm_reward = False

    #model = PPO2(MlpPolicy, env, verbose=0,tensorboard_log="./log_model/", gamma=0.985)
    model = PPO2.load("saves/ppo2.zip",env, verbose=0,tensorboard_log="./log_model2/", gamma=0.985)

This cell allows you to train your agent on a certain number of timesteps. This will also execute a accelerated visualisation of your agent performance. To created your own agent, just remove the last # in the cell above and add them to the two loading functions.

In [ ]:
if __name__ == '__main__':
    model.learn(total_timesteps=20000000,tb_log_name="lavictoiredesesgrandsmorts", reset_num_timesteps=False)

This cell allows you to observe how your agent is behaving in real time. There is no more learning is this phase.

In [ ]:
obs = env.reset()
start_time = time.time()
while 1:
    current_time = time.time()
    elapsed_time = current_time - start_time
    if elapsed_time > 1.0/120:
        start_time=current_time
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        if dones[0]:
            obs = env.reset()

This cell allows you to evaluate the performance of the loaded PPO algorithm (or the one you created) :

In [ ]:
obs = env.reset()
start_time = time.time()
n_step = 1000

catch = [0]
t = [0]
box = [0]

rewards = n_step*[0]
vmax = n_step*[0]
vavg = n_step*[0]
dminf = n_step*[0]
davgf = n_step*[0]
wavgf = n_step*[0]
i = 0

# Performance indicators :
reward = [] # 
vmaxd = [0] #
vavgd = [] #
dmin = [1000000] 
davg = []
wavgd = []
first=True
while 1:
    if i == n_step:
        break

    
    action, _states = model.predict(obs)
    obs, _, dones, _ = env.env_method("evaluate",action.reshape((6,)), reward,vmaxd,vavgd,dmin,davg,wavgd,catch,t,box)[0]
    
    if dones:
        obs = env.reset()
        rewards[i] = np.mean(reward)
        vmax[i] = vmaxd[0]
        vavg[i] = np.mean(vavgd)
        dminf[i] = dmin[0]
        davgf[i] = np.mean(davg)
        wavgf[i] = np.mean(wavgd)
        
        reward = []
        vmaxd = [0]
        vavgd = []
        dmin = [1000000]
        davg = []
        wavgd = []

        i += 1
        if not i%10:
            clear_output(wait=True)
            print(i//10,"%")


In [ ]:
rewards

## SAC agent
This cell lets you load the SAC agent we trained (on 5 millions iterations number).

In [ ]:
import gym
from IPython.display import clear_output
from kendama_env import KendamaEnv
import numpy as np 
import time
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines import PPO2,SAC
from stable_baselines.common.vec_env import VecNormalize
from IPython.display import clear_output

def make_env(rank, seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = KendamaEnv(render=(rank==0))
        return env
    
    return _init

if __name__ == '__main__':
    
    # Uncomment this and comment below if you want to use multiprocessing
    # num_cpu = 4
    # env = SubprocVecEnv([make_env( i) for i in range(num_cpu)])
    # env = VecNormalize(env, norm_reward= False)
    env = KendamaEnv(render=True)
    env = DummyVecEnv([lambda:env])
    
    env = VecNormalize.load("saves/vec_normalize_sac.pkl", env)
    env.norm_reward = False

    #model = SAC(MlpPolicy, env, verbose=0,tensorboard_log="./log_model/", gamma=0.985)
    model = SAC.load("saves/sac.zip",env, verbose=0,tensorboard_log="./log_model2/", gamma=0.985)


This cell allows you to train your agent on a certain number of timesteps. This will also execute a accelerated visualisation of your agent performance. To created your own agent, just remove the last # in the cell above and add them to the two loading functions.

In [ ]:
if __name__ == '__main__':
    model.learn(total_timesteps=20000000,tb_log_name="lavictoiredesesgrandsmorts", reset_num_timesteps=False)

This cell allows you to observe how your agent is behaving in real time. There is no more learning is this phase.

In [ ]:
obs = env.reset()
start_time = time.time()
while 1:
    current_time = time.time()
    elapsed_time = current_time - start_time
    if elapsed_time > 1.0/120:
        start_time=current_time
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        if dones[0]:
            obs = env.reset()


This cell allows you to evaluate the performance of the loaded SAC algorithm (or the one you created) :

In [ ]:
obs = env.reset()
start_time = time.time()
n_step = 1000

catch = [0]
t = [0]
box = [0]

rewards = n_step*[0]
vmax = n_step*[0]
vavg = n_step*[0]
dminf = n_step*[0]
davgf = n_step*[0]
wavgf = n_step*[0]
i = 0

# Performance indicators :
reward = []
vmaxd = [0]
vavgd = []
dmin = [1000000]
davg = []
wavgd = []
first=True
while 1:
    if i == n_step:
        break

    
    action, _states = model.predict(obs)
    obs, _, dones, _ = env.env_method("evaluate",action.reshape((6,)), reward,vmaxd,vavgd,dmin,davg,wavgd,catch,t,box)[0]
    
    if dones:
        obs = env.reset()
        rewards[i] = np.mean(reward)
        vmax[i] = vmaxd[0]
        vavg[i] = np.mean(vavgd)
        dminf[i] = dmin[0]
        davgf[i] = np.mean(davg)
        wavgf[i] = np.mean(wavgd)
        
        reward = []
        vmaxd = [0]
        vavgd = []
        dmin = [1000000]
        davg = []
        wavgd = []

        i += 1
        if not i%10:
            clear_output(wait=True)
            print(i//10,"%")
